In [1]:

import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

print("Torch version: ", torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)


Torch version:  2.5.1+cu121
Device:  cuda


In [2]:

da_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=5, translate=(0.2, 0.2)),
    transforms.ToTensor(),
])

da_test = transforms.Compose([
    transforms.ToTensor(),
])

class CIFAR10_dataset(Dataset):
    def __init__(self, transform, partition="train"):
        print("\nLoading CIFAR10 ", partition, " Dataset...")
        self.partition = partition
        self.transform = transform
        if self.partition == "train":
            self.data = torchvision.datasets.CIFAR10('.data/', train=True, download=True)
        else:
            self.data = torchvision.datasets.CIFAR10('.data/', train=False, download=True)
        print("\tTotal Len.: ", len(self.data), "\n", 50*"-")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx][0]
        image_tensor = self.transform(image)
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()
        return {"img": image_tensor, "label": label}

train_dataset = CIFAR10_dataset(da_train, partition="train")
test_dataset = CIFAR10_dataset(da_test, partition="test")



Loading CIFAR10  train  Dataset...


100%|██████████| 170M/170M [00:04<00:00, 38.1MB/s]


Extracting .data/cifar-10-python.tar.gz to .data/
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------


In [3]:

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Num workers 1


In [4]:

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            ConvBlock(3, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
            ConvBlock(128, 256),
            ConvBlock(256, 512)
        )
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = SimpleCNN(num_classes=10)
print(net)
print("Params: ", sum(p.numel() for p in net.parameters() if p.requires_grad))


SimpleCNN(
  (conv_layers): Sequential(
    (0): ConvBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvBlock(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): ConvBlock(
      (conv): C

In [5]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, min_lr=0.00001)
epochs = 100
net.to(device)

print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0
for epoch in range(epochs):
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:
            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()
            train_loss += loss.item()
    train_loss /= (len(train_dataloader.dataset) / batch_size)
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
        with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
            for batch in tepoch:
                images = batch["img"].to(device)
                labels = batch["label"].to(device)
                outputs = net(images)
                test_loss += criterion(outputs, labels)
                labels = torch.argmax(labels, dim=1)
                pred = torch.argmax(outputs, dim=1)
                test_correct += pred.eq(labels).sum().item()
    lr_scheduler.step(test_loss)
    test_loss /= (len(test_dataloader.dataset) / batch_size)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)
    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
        epoch + 1, train_loss, test_loss, 100. * train_correct / len(train_dataloader.dataset), test_accuracy
    ))
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch
        torch.save(net.state_dict(), "best_model.pt")
print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)



---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:02<00:00, 40.03batch/s]


[Epoch 1] Train Loss: 1.466924 - Test Loss: 1.395232 - Train Accuracy: 45.83% - Test Accuracy: 51.85%


Test 1: 100%|██████████| 100/100 [00:02<00:00, 38.59batch/s]

[Epoch 2] Train Loss: 1.116237 - Test Loss: 0.958854 - Train Accuracy: 59.66% - Test Accuracy: 65.86%


...

[Epoch 98] Train Loss: 0.145465 - Test Loss: 0.417443 - Train Accuracy: 94.80% - Test Accuracy: 87.94%



Test 98: 100%|██████████| 100/100 [00:02<00:00, 34.79batch/s]

[Epoch 99] Train Loss: 0.143514 - Test Loss: 0.416589 - Train Accuracy: 94.89% - Test Accuracy: 87.78%



Test 99: 100%|██████████| 100/100 [00:03<00:00, 32.61batch/s]

[Epoch 100] Train Loss: 0.142463 - Test Loss: 0.418041 - Train Accuracy: 94.96% - Test Accuracy: 87.94%

BEST TEST ACCURACY:  87.94  in epoch  82
